In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np

from shapely import geometry, ops


# Stop Times Full

In [4]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'
stop_times_reduced = pd.read_csv(gtfs_dir + 'stop_times_reduced.csv')

stop_times_reduced['hr'] = np.where(stop_times_reduced['hr']>23, stop_times_reduced['hr'] - 24, stop_times_reduced['hr'])

stop_times_reduced_o = stop_times_reduced.copy()
stop_times_reduced_d = stop_times_reduced.copy()
stop_times_reduced_d['stop_sequence_new'] = stop_times_reduced_d['stop_sequence_new'] - 1

In [5]:
stop_times_od = stop_times_reduced_o.merge(stop_times_reduced_d, left_on = ['trip_id', 'stop_sequence_new'],
                           right_on = ['trip_id', 'stop_sequence_new'], suffixes = ['_o', '_d'])

In [6]:
stop_times_od['min_o'] = stop_times_od['min_o'].apply(lambda x: '{0:0>2}'.format(x))
stop_times_od['min_d'] = stop_times_od['min_d'].apply(lambda x: '{0:0>2}'.format(x))

In [7]:
stop_times_od['hr_o'] = stop_times_od['hr_o'].astype(str)
stop_times_od['hr_d'] = stop_times_od['hr_d'].astype(str)
# stop_times_od['min_o'] = stop_times_od['min_o'].astype(str)

In [8]:
stop_times_od['o_time_str'] = stop_times_od['hr_o'] + ':' + stop_times_od['min_o']
stop_times_od['d_time_str'] = stop_times_od['hr_d'] + ':' + stop_times_od['min_d']

In [9]:
stop_times_od['o_time'] = pd.to_datetime(stop_times_od['o_time_str'], format = '%H:%M')
stop_times_od['d_time'] = pd.to_datetime(stop_times_od['d_time_str'], format = '%H:%M')

stop_times_od['cost'] = ((stop_times_od['d_time'] - stop_times_od['o_time']).dt.seconds)/60

stop_times_od['o_time'] = stop_times_od['o_time'].dt.time
stop_times_od['d_time'] = stop_times_od['d_time'].dt.time

trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']], how = 'inner')

In [11]:
stop_times_od = stop_times_od.drop(columns = ['o_time_str', 'd_time_str', 'o_time', 'd_time'])

In [12]:
stop_times_od.to_csv(gtfs_dir + 'stop_times_full.csv', index = False)

# Stop Times Simple

In [2]:
shapes = pd.read_csv('GTFS/TTC_2016-10-03/shapes.txt')
routes = pd.read_csv('GTFS/TTC_2016-10-03/routes.txt')
trips = pd.read_csv('GTFS/TTC_2016-10-03/trips.txt')
stop_times = pd.read_csv('GTFS/TTC_2016-10-03/stop_times.txt')

In [3]:
stop_times[stop_times['arrival_time'] !=  stop_times['departure_time']]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled


In [4]:
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,32830004,6:02:48,6:02:48,14420,1,NaN,0,0,NaN
1,32830004,6:04:28,6:04:28,14421,2,NaN,0,0,0.5880
2,32830004,6:05:40,6:05:40,14422,3,NaN,0,0,1.0117
3,32830004,6:06:50,6:06:50,14423,4,NaN,0,0,1.4137
4,32830004,6:08:19,6:08:19,14424,5,NaN,0,0,2.0289
...,...,...,...,...,...,...,...,...,...
4917775,32966941,25:07:06,25:07:06,742,12,NaN,0,0,2.7706
4917776,32966941,25:07:33,25:07:33,4769,13,NaN,0,0,2.9306
4917777,32966941,25:08:14,25:08:14,5702,14,NaN,0,0,3.1658
4917778,32966941,25:08:49,25:08:49,9875,15,NaN,0,0,3.3666


In [5]:
stop_times['hr'] = (stop_times['arrival_time'].str.split(':').str[0]).astype(int)
stop_times['min'] = (stop_times['arrival_time'].str.split(':').str[1]).astype(int)
stop_times['sec'] = (stop_times['arrival_time'].str.split(':').str[2]).astype(int)
stop_times_simple = stop_times[['trip_id', 'stop_id', 'stop_sequence', 'hr', 'min','sec']]

In [6]:
stop_times_simple['min'] = np.where(stop_times_simple['sec']>29, stop_times_simple['min'] + 1, stop_times_simple['min'])
stop_times_simple = stop_times_simple[['trip_id', 'stop_id', 'stop_sequence', 'hr', 'min']]
stop_times_simple

/Users/Rick/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,trip_id,stop_id,stop_sequence,hr,min
0,32830004,14420,1,6,3
1,32830004,14421,2,6,4
2,32830004,14422,3,6,6
3,32830004,14423,4,6,7
4,32830004,14424,5,6,8
...,...,...,...,...,...
4917775,32966941,742,12,25,7
4917776,32966941,4769,13,25,8
4917777,32966941,5702,14,25,8
4917778,32966941,9875,15,25,9


In [7]:
stop_times_simple.to_csv('GTFS/TTC_2016-10-03/stop_times_simple.csv', index = False)

In [49]:
stop_times_simple = pd.read_csv('GTFS/TTC_2016-10-03/stop_times_simple.csv')
stop_times_am = stop_times_simple[stop_times_simple['hr'].isin([7,8])]
stop_times_am

,trip_id,stop_id,stop_sequence,hr,min
207,32830010,14435,32,7,1
237,32830011,14433,30,7,0
238,32830011,14434,31,7,3
239,32830011,14435,32,7,6
266,32830012,14430,27,7,0
...,...,...,...,...,...
4917073,32966897,5702,14,8,44
4917074,32966897,9875,15,8,45
4917075,32966897,4479,16,8,46
4917076,32966898,14321,1,8,58


In [50]:
trips_simple = trips[['trip_id', 'route_id', 'trip_headsign', 'shape_id', 'direction_id', 'service_id']].drop_duplicates()
trips_simple = trips_simple.merge(routes[['route_id', 'route_long_name', 'route_short_name']])
trips_simple

,trip_id,route_id,trip_headsign,shape_id,direction_id,service_id,route_long_name,route_short_name
0,32830005,47526,LINE 1 (YONGE-UNIVERSITY),619622,0,1,LINE 1 (YONGE-UNIVERSITY),1
1,32830006,47526,LINE 1 (YONGE-UNIVERSITY),619622,0,1,LINE 1 (YONGE-UNIVERSITY),1
2,32830007,47526,LINE 1 (YONGE-UNIVERSITY),619622,0,1,LINE 1 (YONGE-UNIVERSITY),1
3,32830008,47526,LINE 1 (YONGE-UNIVERSITY),619622,0,1,LINE 1 (YONGE-UNIVERSITY),1
4,32830009,47526,LINE 1 (YONGE-UNIVERSITY),619622,0,1,LINE 1 (YONGE-UNIVERSITY),1
...,...,...,...,...,...,...,...,...
137666,32966940,48181,NORTH - 93 PARKVIEW HILLS TOWARDS PARKVIEW HILLS,628404,1,4,PARKVIEW HILLS,93
137667,32966941,48181,NORTH - 93 PARKVIEW HILLS TOWARDS PARKVIEW HILLS,628404,1,4,PARKVIEW HILLS,93
137668,32949577,47613,SOUTH - 307S BATHURST BLUE NIGHT SHORT TURN,621229,0,4,BATHURST,307
137669,32949578,47613,SOUTH - 307S BATHURST BLUE NIGHT SHORT TURN,621229,0,4,BATHURST,307


In [51]:
trips_simple['route_short_name'] = trips_simple['route_short_name'].astype(int)

## Rebasing `INT_ID` and `trip_id`

In [48]:
ints = pd.read_csv('GTFS/TTC_2016-08-05/GIS Simplify/int_stop.csv')

In [49]:
i = 1000
for index, row in ints.iterrows():
    
    if row['INT_ID'] < 1000:
        ints.at[index, 'int_new'] = row['INT_ID']
    else:
        ints.at[index, 'int_new'] = i
        i = i + 1


In [50]:
ints['int_new'] = ints['int_new'].astype(int)

In [51]:
ints.to_csv('GTFS/TTC_2016-08-05/GIS Simplify/int_stop_reduced_rebase.csv', index = False)

In [52]:
new_trip_id = 1
trips_lookup = []
for row in list(trips_simple['trip_id']):
    trips_lookup.append([new_trip_id, row])
    new_trip_id = new_trip_id + 1

In [53]:
trips_lookup_df = pd.DataFrame.from_records(trips_lookup, columns = ['new_trip_id', 'trip_id'])

In [54]:
trips_lookup_df.to_csv('GTFS/TTC_2016-08-05/GIS Simplify/trips_lookup.csv', index = False)

In [55]:
trips_simple[trips_simple['route_short_name'] == 53][['trip_headsign']].drop_duplicates()

,trip_headsign
88494,EAST - 53B STEELES EAST TOWARDS MARKHAM
88499,EAST - 53E STEELES EAST EXPRESS TOWARDS MARKHAM
88502,EAST - 53F STEELES EAST EXPRESS TOWARDS STAINE...
88546,EAST - 53A STEELES EAST TOWARDS STAINES
88773,WEST - 53 STEELES EAST TOWARDS FINCH STATION
88777,WEST - 53E STEELES EAST EXPRESS TOWARDS FINCH ...
88926,WEST - 53 STEELES EAST TOWARDS FINCH STATION


## Separating out E/F branches

In [52]:
for index, row in trips_simple.iterrows():
    
    if row['route_short_name'] < 4:
        continue
    
    express = row['trip_headsign'].split(' ')[2]
    
    if express[-1] != 'E':
        if express != '53F':
            if express != '60F':#specific exception for steeles
                continue
    
    trips_simple.at[index, 'route_short_name'] = row['route_short_name'] + 9000

In [53]:
trips_simple[trips_simple['route_short_name']>1000][['route_short_name', 'trip_headsign']].drop_duplicates()

,route_short_name,trip_headsign
10967,9112,SOUTH - 112E WEST MALL EXPRESS TOWARDS KIPLING...
14254,9012,WEST - 12E KINGSTON RD EXPRESS TOWARDS VICTORI...
20255,9131,WEST - 131E NUGGET EXPRESS TOWARDS SCARBOROUGH...
20430,9131,EAST - 131E NUGGET EXPRESS TOWARDS OLD FINCH/M...
20431,9131,EAST - 131E NUGGET EXPRESS TOWARDS KENNEDY STA...
42842,9024,SOUTH - 24E VICTORIA PARK TOWARDS VICTORIA PAR...
43091,9024,NORTH - 24E VICTORIA PARK TOWARDS STEELES
66101,9041,SOUTH - 41E KEELE EXPRESS TOWARDS KEELE STATION
66238,9041,NORTH - 41E KEELE EXPRESS TOWARDS YORK UNIVERSITY
70225,9045,SOUTH - 45E KIPLING EXPRESS TOWARDS KIPLING ST...


In [54]:
trips_weekday = trips_simple[trips_simple['service_id'] == 1].copy()

In [56]:
trips_weekday.to_csv('GTFS/TTC_2016-10-03/trips_weekday.csv')

## Creating a Stop String Lookup

In [60]:
intersections = gpd.read_file('GIS/major_minor_int.geojson')

In [61]:
stn = pd.read_csv('GIS/stations.csv')

In [62]:
directory = intersections[['INT_ID', 'INTERSEC5']]
directory = directory.drop_duplicates()
directory = directory.rename(columns = {'INTERSEC5':'string'})

In [63]:
stn = stn.rename(columns = {'name':'string'})

In [64]:
directory = directory.append(stn[['INT_ID', 'string']])

In [65]:
directory['string'] = directory['string'].str.title()

In [66]:
directory = directory.merge(ints['INT_ID'])

In [67]:
directory.to_csv('GIS/int_directory.csv', index = False)